# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [2]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
import tensorflow as tf
import numpy as np
import string

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [22]:

max_tries = 6
max_epochs = max_tries
c2i = {letter: idx for idx, letter in enumerate(string.ascii_lowercase + '_ ')}
max_input_len = 35
alhabets_len = 26

In [32]:
def encode_word(masked_word, incorrect_guesses):
    masked_word = incorrect_guesses.ljust(max_tries, ' ') + masked_word.rjust(max_input_len - max_tries, ' ')
    encoded = [c2i.get(char) for char in masked_word]
    return np.array(encoded)

In [54]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        self.model = tf.keras.models.load_model("model_checkpoint_h24.keras")
        self.alphabet_list = string.ascii_lowercase
        self.incorrect_guesses = []
        
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word):
        
        if len(self.guessed_letters)>0:
            recent_guessed = self.guessed_letters[-1]
            if recent_guessed not in word:
                self.incorrect_guesses.append(recent_guessed)
        print(self.incorrect_guesses)
        
        incorrect_guesses_str = "".join(self.incorrect_guesses)
        
        encoded = encode_word(word.replace(" ",""),incorrect_guesses_str)
        probabilities = self.model.predict(np.array([encoded]))[0]
        
        for idx, letter in enumerate(self.alphabet_list):
            if letter in self.guessed_letters:
                probabilities[idx] = -1

        guess = string.ascii_lowercase[np.argmax(probabilities)]

        return guess
    

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        self.incorrect_guesses=[]
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [55]:
api = HangmanAPI(access_token="1721821c3df8eccf4b64f490d82547", timeout=2000)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [56]:
import time
for i in range(100):
    
    api.start_game(practice=1,verbose=True)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    practice_success_rate = total_practice_successes / total_practice_runs
    print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))
    time.sleep(4.5)

Successfully start a new game! Game ID: c785db8d4fff. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ .
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Guessing letter: e
Sever response: {'game_id': 'c785db8d4fff', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ e e _ _ _ _ _ '}
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Guessing letter: r
Sever response: {'game_id': 'c785db8d4fff', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ e e _ _ r _ _ '}
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Guessing letter: a
Sever response: {'game_id': 'c785db8d4fff', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ e e _ _ r a _ '}
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Guessing letter: l
Sever response: {'game_id': 'c785db8d4fff', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ e e l _ r a _ '}
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Guessing letter: t
Sever response: {'game_id': 'c785db8d4fff', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ t e e l _ r a _ '}
[]
1/1 ━━━━━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Guessing letter: u
Sever response: {'game_id': '10a118492958', 'status': 'ongoing', 'tries_remains': 1, 'word': '_ o u r l o o k i n g '}
['e', 't', 'a', 'p', 'b']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Guessing letter: f
Sever response: {'game_id': '10a118492958', 'status': 'failed', 'tries_remains': 0, 'word': '_ o u r l o o k i n g '}
Failed game: 10a118492958. Because of: # of tries exceeded!
run 177 practice games out of an allotted 100,000. practice success rate so far = 0.299
Successfully start a new game! Game ID: 63bedd3f85b5. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ _ .
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Guessing letter: e
Sever response: {'game_id': '63bedd3f85b5', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ _ _ _ _ _ e '}
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Guessing letter: t
Sever response: {'game_id': '63bedd3f85b5', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ t _ _ _ _ _ t _ _ _ e '}

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Guessing letter: r
Sever response: {'game_id': '1788e8e4290a', 'status': 'failed', 'tries_remains': 0, 'word': '_ _ i _ n e s s '}
Failed game: 1788e8e4290a. Because of: # of tries exceeded!
run 180 practice games out of an allotted 100,000. practice success rate so far = 0.300
Successfully start a new game! Game ID: fa6ad2fe0a3a. # of tries remaining: 6. Word: _ _ _ _ _ _ .
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Guessing letter: e
Sever response: {'game_id': 'fa6ad2fe0a3a', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ '}
['e']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Guessing letter: a
Sever response: {'game_id': 'fa6ad2fe0a3a', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ _ _ '}
['e', 'a']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Guessing letter: o
Sever response: {'game_id': 'fa6ad2fe0a3a', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ o _ _ _ _ '}
['e', 'a']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Guessing letter: r

Sever response: {'game_id': 'd47f98bd89ad', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ e _ _ _ _ _ '}
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Guessing letter: r
Sever response: {'game_id': 'd47f98bd89ad', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ r e _ r _ _ _ '}
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Guessing letter: a
Sever response: {'game_id': 'd47f98bd89ad', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ r e _ r _ _ _ '}
['a']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Guessing letter: t
Sever response: {'game_id': 'd47f98bd89ad', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ r e t r _ _ _ '}
['a']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Guessing letter: o
Sever response: {'game_id': 'd47f98bd89ad', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ r e t r _ _ _ '}
['a', 'o']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Guessing letter: i
Sever response: {'game_id': 'd47f98bd89ad', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ r e t r i _ _ '}
['a', 'o

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Guessing letter: h
Sever response: {'game_id': '481972c03975', 'status': 'success', 'tries_remains': 2, 'word': 't e l e n c e p h a l o n s '}
Successfully finished game: 481972c03975
run 186 practice games out of an allotted 100,000. practice success rate so far = 0.296
Successfully start a new game! Game ID: 24e0e6978ffc. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ .
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Guessing letter: e
Sever response: {'game_id': '24e0e6978ffc', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ e _ _ '}
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Guessing letter: s
Sever response: {'game_id': '24e0e6978ffc', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ e _ s '}
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Guessing letter: r
Sever response: {'game_id': '24e0e6978ffc', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ e r s '}
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Guessing letter: t

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Guessing letter: t
Sever response: {'game_id': '3548ab5a2b07', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ a _ _ _ _ _ '}
['e', 'r', 't']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Guessing letter: l
Sever response: {'game_id': '3548ab5a2b07', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ a _ _ _ _ _ '}
['e', 'r', 't', 'l']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Guessing letter: s
Sever response: {'game_id': '3548ab5a2b07', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ a _ _ _ _ s '}
['e', 'r', 't', 'l']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Guessing letter: n
Sever response: {'game_id': '3548ab5a2b07', 'status': 'ongoing', 'tries_remains': 1, 'word': '_ a _ _ _ _ s '}
['e', 'r', 't', 'l', 'n']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Guessing letter: m
Sever response: {'game_id': '3548ab5a2b07', 'status': 'failed', 'tries_remains': 0, 'word': '_ a _ _ _ _ s '}
Failed game: 3548ab5a2b07. Because of: # of tries exceeded!
run 259 practice 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Guessing letter: i
Sever response: {'game_id': '10c5b9956d7a', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ e _ _ _ e _ _ r _ _ i _ '}
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Guessing letter: a
Sever response: {'game_id': '10c5b9956d7a', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ e _ _ _ e _ _ r _ _ i _ '}
['a']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Guessing letter: o
Sever response: {'game_id': '10c5b9956d7a', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ e _ o _ e _ _ r o _ i _ '}
['a']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Guessing letter: s
Sever response: {'game_id': '10c5b9956d7a', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ e _ o _ e _ _ r o s i s '}
['a']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Guessing letter: t
Sever response: {'game_id': '10c5b9956d7a', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ e _ o _ e _ _ r o s i s '}
['a', 't']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Guessing letter: c
Sever respon

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Guessing letter: a
Sever response: {'game_id': '4ca469b5c8e7', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ i l a _ _ i '}
['e']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Guessing letter: t
Sever response: {'game_id': '4ca469b5c8e7', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ i l a _ t i '}
['e']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Guessing letter: n
Sever response: {'game_id': '4ca469b5c8e7', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ i l a n t i '}
['e']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Guessing letter: r
Sever response: {'game_id': '4ca469b5c8e7', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ i l a n t i '}
['e', 'r']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Guessing letter: c
Sever response: {'game_id': '4ca469b5c8e7', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ _ i l a n t i '}
['e', 'r', 'c']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Guessing letter: m
Sever response: {'game_id': '4ca469b5c8e7',

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Guessing letter: a
Sever response: {'game_id': 'e4884488e2c6', 'status': 'ongoing', 'tries_remains': 1, 'word': '_ e r _ _ e _ '}
['s', 't', 'd', 'i', 'a']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Guessing letter: o
Sever response: {'game_id': 'e4884488e2c6', 'status': 'ongoing', 'tries_remains': 1, 'word': '_ e r o _ e _ '}
['s', 't', 'd', 'i', 'a']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Guessing letter: v
Sever response: {'game_id': 'e4884488e2c6', 'status': 'failed', 'tries_remains': 0, 'word': '_ e r o _ e _ '}
Failed game: e4884488e2c6. Because of: # of tries exceeded!
run 269 practice games out of an allotted 100,000. practice success rate so far = 0.290
Successfully start a new game! Game ID: 58631f8880d8. # of tries remaining: 6. Word: _ _ _ _ _ _ _ .
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Guessing letter: e
Sever response: {'game_id': '58631f8880d8', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ '}
['e']
1/1 ━━━━━━━━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Guessing letter: c
Sever response: {'game_id': '9287dbd0391b', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ c _ _ c i _ _ _ i _ '}
['e']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Guessing letter: a
Sever response: {'game_id': '9287dbd0391b', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ c a _ c i _ _ _ i _ '}
['e']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Guessing letter: n
Sever response: {'game_id': '9287dbd0391b', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ c a _ c i _ _ n i n '}
['e']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Guessing letter: r
Sever response: {'game_id': '9287dbd0391b', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ r _ c a _ c i _ _ n i n '}
['e']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Guessing letter: o
Sever response: {'game_id': '9287dbd0391b', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ r o c a _ c i _ o n i n '}
['e']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Guessing let

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
['e']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['e', 'r']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
['e', 'r', 't']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['e', 'r', 't']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e', 'r', 't']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e', 'r', 't', 'i']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e', 'r', 't', 'i', 'd']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Playing  1  th game
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['l']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['l']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['l']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['l']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['l']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['l', 'u']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['l', 'u', 'd']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['l', 'u', 'd']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['l', 'u', 'd', 'p']
1/1 ━━━━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e', 'a', 'o', 'l']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e', 'a', 'o', 'l']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e', 'a', 'o', 'l', 'r']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['e', 'a', 'o', 'l', 'r']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['e', 'a', 'o', 'l', 'r']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Playing  10  th game
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
['e', 'l']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
['e', 'l']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
['e', 'l', 'i']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['e', 'l', 'i']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e', 'l', 'i', 'r']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['e', 'l', 'i', 'r', 's']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Playing  11  th game
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 1

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['e', 't', 'm']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['e', 't', 'm', 'd']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e', 't', 'm', 'd']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e', 't', 'm', 'd']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e', 't', 'm', 'd', 'y']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e', 't', 'm', 'd', 'y']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Playing  20  th game
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['t']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['t']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
['t']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['t']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['t', 'p']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['t', 'p', 'c']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Playing  21  th game
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['e']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['e']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e', 'r']
1/1 ━━━━━━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['s']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['s']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['s', 't']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['s', 't', 'l']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['s', 't', 'l']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['s', 't', 'l', 'i']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['s', 't', 'l', 'i', 'a']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['s', 't', 'l', 'i', 'a']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['s', 't', 'l', 'i', 'a']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['s', 't', 'l', 'i', 'a']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Playing  30  th game
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['r']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['r']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['r']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['r', 'm']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['r']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['r']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['r', 't']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
['r', 't']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['r', 't']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
['r', 't']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['r', 't']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Playing  39  th game
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['s']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['s']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['s']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['s']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['s', 'd']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['s', 'd']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
['s', 'd']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
['s', 'd']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['s', 'd', 'c']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['s', 'd', 'c', 'f']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['s', 'r', 't', 'i']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['s', 'r', 't', 'i', 'o']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['s', 'r', 't', 'i', 'o']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Playing  495  th game
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e', 'a']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e', 'a']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e', 'a', 'r']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e', 'a', 'r', 'c']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e', 'a', 'r', 'c', 'n']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Playing  496  th game
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['e']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['e', 'i']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e', 'i']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['e', 'i']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
['e', 'i', 't']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['e', 'i', 't']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['e', 'i', 't', 'm']
1/1 ━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['n']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['n']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['n', 't']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['n', 't', 'v']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['n', 't', 'v', 'd']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['n', 't', 'v', 'd', 'x']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['n', 't', 'v', 'd', 'x']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['n', 't', 'v', 'd', 'x']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['n', 't', 'v', 'd', 'x']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Playing  505  th game
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['r']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['r', 's']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['r', 's', 't']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['r', 's', 't']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['r', 's', 't', 'd']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['r', 's', 't', 'd', 'm']
1/1 ━━━━━━━━━━━━━━━━━━━━ 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['r', 'l', 's', 'm', 'd']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['r', 'l', 's', 'm', 'd']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Playing  514  th game
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['t']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['t', 'r']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['t', 'r']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['t', 'r']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['t', 'r']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['t', 'r']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['t', 'r']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
['t', 'r']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['t', 'r']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Playing  515  th game
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
['e']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['e']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['e', 'n']
1/1 ━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['v', 'c']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['v', 'c', 'o']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['v', 'c', 'o', 'g']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['v', 'c', 'o', 'g']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Playing  522  th game
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
['e']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
['e', 'a']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
['e', 'a']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['e', 'a', 'l']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
['e', 'a', 'l']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['e', 'a', 'l']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['e', 'a', 'l']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['e', 'a', 'l', 't']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e', 'a', 'l', 't', 'v']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Playing  523  th game
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
['e']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['e', 'n']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/st

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
[]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['p']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [ ]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)